In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
septiembre = pd.read_parquet('/content/septiembre_ml_1.parquet')
agosto = pd.read_parquet('/content/agosto_ml_1.parquet')
julio = pd.read_parquet('/content/julio_ml_1.parquet')
junio = pd.read_parquet('/content/junio_ml_1.parquet')
mayo = pd.read_parquet('/content/abril_ml_1.parquet')
abril = pd.read_parquet('/content/abril_ml_1.parquet')
#marzo = pd.read_parquet('/taxi_filtrado3.snappy.parquet')
#febrero= pd.read_parquet('/taxi_filtrado2.snappy.parquet')
#enero= pd.read_parquet('/taxi_filtrado1.snappy.parquet')

In [10]:
viajes_taxis = pd.concat([septiembre,agosto, julio, junio, mayo, abril])

In [11]:
viajes_taxis.describe()

,trip_distance,total_amount,hora_del_dia,dia_de_la_semana
count,1.583732e+07,1.583732e+07,1.583732e+07,1.583732e+07
mean,1.980092e+00,2.183878e+01,1.426012e+01,3.034799e+00
std,1.423933e+00,1.020181e+01,5.811822e+00,1.919423e+00
min,0.000000e+00,-5.750000e+00,0.000000e+00,0.000000e+00
25%,9.900000e-01,1.540000e+01,1.100000e+01,1.000000e+00
50%,1.600000e+00,1.965000e+01,1.500000e+01,3.000000e+00
75%,2.570000e+00,2.590000e+01,1.900000e+01,5.000000e+00
max,7.420000e+00,7.387800e+02,2.300000e+01,6.000000e+00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
import matplotlib.pyplot as plt

# Cargar el DataFrame
# Asegúrate de tener 'viajes_taxis' cargado con los datos relevantes

# Seleccionar características y variable objetivo
features_regresion = ['trip_distance', 'hora_del_dia', 'dia_de_la_semana']
target_regresion = 'total_amount'

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(viajes_taxis[features_regresion], viajes_taxis[target_regresion], test_size=0.1, random_state=42)

# Modelos
modelos = {
    #'RandomForest': RandomForestRegressor(n_estimators=120, random_state=42, max_depth = 12),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=150, random_state=42, learning_rate = 0.05),
    # 'AdaBoost': AdaBoostRegressor(n_estimators=100, random_state=42),
    # 'KNeighbors': KNeighborsRegressor(n_neighbors=3)
}

# Entrenar y guardar cada modelo
for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    joblib.dump(modelo, f'{nombre}_model.joblib')

    # Hacer predicciones
    y_pred = modelo.predict(X_test)

    # Calcular el Mean Squared Error (MSE)
    mse = mean_squared_error(y_test, y_pred)
    print(f'{nombre} Mean Squared Error: {mse}')

    # Calcular el Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'{nombre} Mean Absolute Error: {mae}')

    y_train_pred = modelo.predict(X_train)

    # Calcular el Mean Squared Error (MSE)
    mse_train = mean_squared_error(y_train, y_train_pred)
    print(f'{nombre} Mean Squared Error (Train): {mse_train}')

    # Calcular el Mean Absolute Error (MAE)
    mae_train = mean_absolute_error(y_train, y_train_pred)
    print(f'{nombre} Mean Absolute Error (Train): {mae_train}')

    # Crear una serie de números para el eje x (1 a 200)
    n_valores = 150
    eje_x = range(1, n_valores + 1)

    # Obtener solo los primeros 200 valores
    y_test_subset = y_test[:n_valores]
    y_pred_subset = y_pred[:n_valores]

    # Graficar las líneas para y_test y y_pred
    plt.figure(figsize=(10, 6))
    plt.plot(eje_x, y_test_subset, label='Valores Reales', marker='o', linestyle='-', color='blue')
    plt.plot(eje_x, y_pred_subset, label='Valores Predichos', marker='o', linestyle='-', color='red')

    # Configurar el gráfico
    plt.title(f'Comparación entre Valores Reales y Valores Predichos ({nombre})')
    plt.xlabel('Número de Valores')
    plt.ylabel('Valores')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'plot_{nombre}.png')
    plt.show()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
import matplotlib.pyplot as plt
import seaborn as sns


# Realizar validación cruzada
for nombre, modelo in modelos.items():

    modelo = joblib.load('GradientBoosting_model (2).joblib')
    # Utilizar cross_val_predict para obtener predicciones en validación cruzada
    y_pred_cv = cross_val_predict(modelo, X_train, y_train, cv=5)

    # Calcular el Mean Squared Error (MSE) en validación cruzada
    mse_cv = mean_squared_error(y_train, y_pred_cv)
    print(f'{nombre} Cross-Validated Mean Squared Error: {mse_cv}')

    # Calcular el Mean Absolute Error (MAE) en validación cruzada
    mae_cv = mean_absolute_error(y_train, y_pred_cv)
    print(f'{nombre} Cross-Validated Mean Absolute Error: {mae_cv}')

    # Graficar un histograma de la distribución del error entre valores predichos y reales
    error = y_pred_cv - y_train
    plt.figure(figsize=(10, 6))
    sns.histplot(error, kde=True, color='blue')  # Utilizamos seaborn para el histograma
    plt.title(f'Distribución del Error entre Valores Predichos y Reales ({nombre})')
    plt.xlabel('Error')
    plt.ylabel('Frecuencia')
    plt.grid(True)
    plt.savefig(f'histogram_{nombre}.png')
    plt.show()

In [ ]:
# Calcular el error entre las predicciones y los valores reales
error = y_test - y_pred

# Crear un histograma del error con límites en el eje x entre 0 y 100
plt.figure(figsize=(10, 6))
plt.hist(error, bins=30, range=(-60, 60), edgecolor='black', alpha=0.7)
plt.title('Distribución del Error entre Valores Predichos y Reales de la prediccion del precio')
plt.xlabel('Error')
plt.ylabel('Frecuencia')
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
import matplotlib.pyplot as plt
import seaborn as sns  # Agregamos seaborn para el histograma

# ... (resto del código)

y_pred_cv = cross_val_predict(modelo_cargado, X_train, y_train, cv=5)

# Calcular el Mean Squared Error (MSE) en validación cruzada
mse_cv = mean_squared_error(y_train, y_pred_cv)
print(f'{nombre} Cross-Validated Mean Squared Error: {mse_cv}')

    # Calcular el Mean Absolute Error (MAE) en validación cruzada
mae_cv = mean_absolute_error(y_train, y_pred_cv)
print(f'{nombre} Cross-Validated Mean Absolute Error: {mae_cv}')

    # Graficar un histograma de la distribución del error entre valores predichos y reales
error = y_pred_cv - y_train
plt.figure(figsize=(10, 6))
sns.histplot(error, kde=True, color='blue')  # Utilizamos seaborn para el histograma
plt.title(f'Distribución del Error entre Valores Predichos y Reales ({nombre})')
plt.xlabel('Error')
plt.ylabel('Frecuencia')
plt.grid(True)
plt.savefig(f'histogram_{nombre}.png')
plt.show()

# ... (resto del código)
